In [1]:
from textwrap import wrap
import pysam
import pandas as pd
from tqdm import tqdm

In [2]:
CHUNK_LEN = 1024
OVERLAP_BP = 128
MAX_SEQ_LENGTH = 5000

In [3]:
workdir = '/lustre/groups/epigenereg01/workspace/projects/vale/'

In [5]:
input_fa = workdir + 'mlm/fasta/241_mammals.shuffled.fa'
input_seqs = pd.read_csv(input_fa + '.fai', sep='\t', header=None, usecols=[0])[0].squeeze().values

input_fasta = pysam.FastaFile(input_fa)

In [6]:
output_fa = workdir + f'mlm/dnabert2-3utr/fasta/chunk_{CHUNK_LEN}_overlap_{OVERLAP_BP}.fa'

In [7]:
with open(output_fa,'w') as fout:
    for seq_name in tqdm(input_seqs):
        seq = input_fasta.fetch(seq_name).rstrip().replace('-','')
        seq = seq[:MAX_SEQ_LENGTH]
        seq_name = ':'.join(seq_name.split(':')[:-1]) #remove seq length at the end
        for chunk_idx,start_idx in enumerate(range(0,len(seq),CHUNK_LEN-OVERLAP_BP)):
            chunk = seq[start_idx:start_idx+CHUNK_LEN]
            fout.write(f'>{seq_name}:{chunk_idx}\n') #add chunk idx at the end
            for line in wrap(chunk, 80): #wrap sequence with standard FASTA width
                fout.write(line+'\n')

100%|██████████| 3783714/3783714 [12:33<00:00, 5020.68it/s]


In [8]:
! samtools faidx {output_fa}